In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

In [7]:
# Carregando os dados do CSV
data = pd.read_csv("Leads.csv")

# Convertendo a coluna target "sim"/"não" para 1/0
data["converteu_em_cliente"] = data["converteu_em_cliente"].map({"não": 0, "sim": 1})

# - paginas_visitadas é a variável mais promissora (representa engajamento)
# - setor_empresa e cidade_lead podem ajudar a explicar conversões
# - id_visitante é apenas identificador e não será usado
features = ["paginas_visitadas", "setor_empresa", "cidade_lead"]
X = data[features]
y = data["converteu_em_cliente"]  # Target: 0 = não converteu, 1 = converteu

# Separando colunas numéricas e categóricas
colunas_numericas = ["paginas_visitadas"]
colunas_categoricas = ["setor_empresa", "cidade_lead"]

In [8]:
# Pré-processamento
# - Normaliza variáveis numéricas para melhorar o desempenho da regressão
# - Codifica variáveis categóricas usando OneHotEncoder
preprocessador = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), colunas_numericas),
        ("cat", OneHotEncoder(handle_unknown='ignore'), colunas_categoricas)
    ]
)

# Criando o pipeline do modelo
# - Aplica pré-processamento e treina a Regressão Logística
modelo = Pipeline(steps=[
    ("preprocessador", preprocessador),
    ("classificador", LogisticRegression(random_state=42))
])


In [9]:
# Dividindo os dados em treino e teste
# 80% dos dados serão usados para treinar, 20% para testar
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinando o modelo
modelo.fit(X_treino, y_treino)

# Fazendo previsões no conjunto de teste
previsoes = modelo.predict(X_teste)

# Avaliando o modelo
print("=== Avaliação do Modelo ===")
print("Acurácia:", accuracy_score(y_teste, previsoes))
# Acurácia: porcentagem de previsões corretas. Indica, de forma geral,
# quantos leads foram corretamente classificados como convertidos ou não.

print("Precisão:", precision_score(y_teste, previsoes))
# Precisão: entre os leads que o modelo previu como convertidos,
# quantos realmente se converteram. Alta precisão significa menos falsos positivos.

print("Recall:", recall_score(y_teste, previsoes))
# Recall: entre todos os leads que realmente se converteram,
# quantos o modelo conseguiu identificar. Alto recall significa menos falsos negativos.

print("\nRelatório detalhado:\n")
print(classification_report(y_teste, previsoes))
# Mostra métricas detalhadas por classe: precisão, recall, f1-score e suporte



=== Avaliação do Modelo ===
Acurácia: 0.895
Precisão: 0.8709677419354839
Recall: 0.6136363636363636

Relatório detalhado:

              precision    recall  f1-score   support

           0       0.90      0.97      0.94       156
           1       0.87      0.61      0.72        44

    accuracy                           0.90       200
   macro avg       0.89      0.79      0.83       200
weighted avg       0.89      0.90      0.89       200



In [10]:
resultado = X_teste.copy()
resultado["previsao_conversao"] = previsoes
resultado["real_conversao"] = y_teste.values
resultado.to_csv("previsoes_leads.csv", index=False)
print("As previsões foram salvas em 'previsoes_leads.csv'.")

As previsões foram salvas em 'previsoes_leads.csv'.
